In [18]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.runnables import RunnablePassthrough, RunnableParallel

from langchain_core.output_parsers import StrOutputParser

from langchain.retrievers import ParentDocumentRetriever

from langchain.storage import InMemoryStore

In [4]:
import os

In [ ]:
os.environ["GOOGLE_API_KEY"] = "api_key_here"

In [7]:
#Load dos modelos (Embedding e LLM)

embedding_model = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", max_tokens=500)

In [8]:
#Carregar o PDF

pdf_link = "os-sertoes.pdf"

loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()

In [9]:
len(pages)

658

In [10]:
#Separar em Chunks

child_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000, 
    chunk_overlap=200,
    length_function=len,
    add_start_index=True
)

In [12]:
#Storages

store = InMemoryStore()

vectorStore = Chroma(embedding_function=embedding_model, persist_directory="childVectorDB")

In [14]:
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorStore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter
)

parent_document_retriever.add_documents(pages, ids=None)

In [21]:
TEMPLATE = """"
    Você é um especialista em literatura brasileira. Responda a pergunta com base no contexto fornecido.
    Query:
    {question}

    Context:
    {context}
"""

rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)

In [19]:
setup_retrival = RunnableParallel({"question": RunnablePassthrough(), "context": parent_document_retriever})

output_parser = StrOutputParser()

In [22]:
parent_chain_retrival = setup_retrival | rag_prompt | llm | output_parser

In [24]:
# Pergunta a serem respondidas sobre "Os Sertões" de Euclides da Cunha:
# 
#1. Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?
#2. Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?
#3. Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?
#4. Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?
#5. Quais são os principais aspectos da crítica social e política presentes em "Os Sertões"? Como esses aspectos refletem a visão do autor sobre o Brasil da época?

parent_chain_retrival.invoke("Quais são os principais aspectos da crítica social e política presentes em 'Os Sertões'? Como esses aspectos refletem a visão do autor sobre o Brasil da época?")

'O excerto fornecido não oferece detalhes suficientes sobre a crítica social e política em "Os Sertões" para uma resposta completa.  O texto menciona a obra como um retrato complexo da realidade geográfica e humana do sertão,  uma mistura de gêneros literários que explica cientificamente e literariamente um momento da história nacional.  Também há uma citação de uma crítica literária de Araripe Júnior, que, embora elogie a obra, demonstra uma leitura equivocada do sertão, do sertanejo e de Antônio Conselheiro, possivelmente influenciada pelo cosmopolitismo da Belle Époque.\n\nPara responder adequadamente à pergunta sobre os principais aspectos da crítica social e política em "Os Sertões" e como eles refletem a visão de Euclides da Cunha sobre o Brasil da época, é necessário o acesso ao conteúdo completo da obra.  O excerto apenas oferece um vislumbre de algumas interpretações críticas, sem detalhar as próprias análises sociais e políticas de Cunha.  A análise completa precisaria aborda